In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable


In [0]:
%run /Workspace/Users/amank0639@gmail.com/fmcg_repo/consolidated_pipeline/1_Setup/utilities

In [0]:
dbutils.widgets.text("Catalog","fmcg")
dbutils.widgets.text("Data_Source","products")

In [0]:
catalog=dbutils.widgets.get("Catalog")
data_source=dbutils.widgets.get("Data_Source")

In [0]:
base_path=f"s3://sportsbar-bucket/{data_source}"


In [0]:

products_raw_df=spark.read.format("csv").option("header",True).option("inferSchema",True).load(base_path)\
            .select("*",F.current_timestamp().alias("read_timestamp"),F.col("_metadata.file_name").alias("file_name")\
                ,F.col("_metadata.file_size").alias("file_size"))
products_raw_df.printSchema()
display(products_raw_df)

products_raw_df.write.format("delta").mode("overwrite").option("enableChangeDataFeed","true")\
    .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")


In [0]:
products_bronze_df=spark.read.table(f"{catalog}.{bronze_schema}.{data_source}")
display(products_bronze_df)

In [0]:
cleaned_df=(
    products_bronze_df
    .dropDuplicates(subset=["product_id"])
    .withColumn("category",F.initcap(F.col("category")))
    .withColumn("variant",F.split("product_name",r"\(")[1])     
    .withColumn("variant",F.regexp_replace("variant",r"\)",""))
    .withColumn("product_name",F.regexp_replace("product_name","(?i)protien","Protein"))
    .withColumn("category",F.regexp_replace("category","(?i)protien","Protein"))
    )
display(cleaned_df)